<a href="https://colab.research.google.com/github/thanhdangg/Knowledge-Distillation-for-Object-Detection/blob/main/Distillation_model_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# Check if GPU is available, and if not, use the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
import torch.nn.functional as F
from torchvision.datasets import VOCDetection
from torchvision import transforms
import torch
from torch.utils.data import DataLoader

In [3]:
student_model = torch.hub.load("ultralytics/yolov5", "yolov5s")

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-8-31 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [4]:
!pip install dill

In [5]:
teacher_model = torch.hub.load("ultralytics/yolov5", "yolov5x6")
teacher_model.eval()

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-8-31 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5x6 summary: 574 layers, 140730220 parameters, 0 gradients, 209.6 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 80, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(160, 160, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(80, 80, kernel_size=(1, 1), stride=(1, 1

In [6]:
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
])

In [7]:
# Load the VOC2012 dataset
train_dataset = VOCDetection(root='./data', year='2012', image_set='train', download=True, transform=transform)
val_dataset = VOCDetection(root='./data', year='2012', image_set='val', download=True, transform=transform)

Using downloaded and verified file: ./data/VOCtrainval_11-May-2012.tar
Extracting ./data/VOCtrainval_11-May-2012.tar to ./data
Using downloaded and verified file: ./data/VOCtrainval_11-May-2012.tar
Extracting ./data/VOCtrainval_11-May-2012.tar to ./data


In [8]:
def custom_collate_fn(batch):
    # Loại bỏ các phần tử None
    batch = [b for b in batch if b is not None]

    # Tạo một danh sách chứa các ảnh và nhãn riêng biệt
    images = torch.stack([item[0] for item in batch], dim=0)
    targets = [item[1] for item in batch]

    return images, targets


In [9]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, collate_fn=custom_collate_fn)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2, collate_fn=custom_collate_fn)


In [10]:
import numpy as np

In [11]:
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
from PIL import UnidentifiedImageError  # Import the exception class


def train(model, train_loader, epochs, learning_rate, device):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    model.train()

    for epoch in range(epochs):
        running_loss = 0.0

        for batch in train_loader:
          try:
            images, targets = batch
            images = images.to(device)

            if images.size(0) == 0:
                continue  # Skip empty batch

            # YOLOv5 returns a list of outputs
            outputs = model(images)[0]  # Select the first output

            # Debugging: Print type and content of targets
            print(f"Type of targets: {type(targets)}")
            print(f"Content of targets: {targets}")

            # Extract numeric data from targets
            if isinstance(targets, dict):
                # Assuming the dictionary contains tensors or lists of tensors
                target_values = []
                for key, value in targets.items():
                    if isinstance(value, (list, np.ndarray, torch.Tensor)):
                        # Convert to numpy array if it's a tensor
                        if isinstance(value, torch.Tensor):
                            value = value.numpy()
                        target_values.append(value)
                    else:
                        print(f"Unexpected type in targets dictionary for key {key}: {type(value)}")

                if len(target_values) == 0:
                    print("No valid numeric data found in targets.")
                    continue

                # Flatten the list of arrays/tensors if necessary
                target_values = np.concatenate([np.array(v) for v in target_values], axis=0)
                targets_tensor = torch.tensor(target_values, dtype=torch.float).to(device)
            else:
                # Handle if targets is not a dictionary
                if isinstance(targets, (list, np.ndarray, torch.Tensor)):
                    # Convert to tensor directly
                    if isinstance(targets, torch.Tensor):
                        targets_tensor = targets.to(device)
                    else:
                        targets_tensor = torch.tensor(targets, dtype=torch.float).to(device)
                elif isinstance(targets, dict):
                    print(f"Unexpected dictionary type for targets: {targets}")
                    # Handle the dictionary case, e.g., extract relevant values
                    # targets_tensor = handle_dict(targets)
                else:
                    print(f"Unexpected type for targets: {type(targets)}")
                    continue


            # Compute the loss
            loss_fn = nn.MSELoss()  # Example loss function
            loss = loss_fn(outputs, targets_tensor)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
          except UnidentifiedImageError as e:
            print(f"Warning: Failed to load image: {e}")
            continue
          except Exception as e:
            print(f"Error during training: {e}")
            continue

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")


In [12]:
epochs = 5
learning_rate = 0.001

In [13]:
train(teacher_model, train_loader, epochs, learning_rate, device)

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:865: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Type of targets: <class 'list'>
Content of targets: [{'annotation': {'filename': '2010_005229.jpg', 'folder': 'VOC2012', 'object': [{'name': 'dog', 'bndbox': {'xmax': '282', 'xmin': '62', 'ymax': '400', 'ymin': '83'}, 'difficult': '0', 'occluded': '0', 'pose': 'Unspecified', 'truncated': '1'}], 'segmented': '0', 'size': {'depth': '3', 'height': '400', 'width': '293'}, 'source': {'annotation': 'PASCAL VOC2010', 'database': 'The VOC2010 Database', 'image': 'flickr'}}}, {'annotation': {'filename': '2010_002734.jpg', 'folder': 'VOC2012', 'object': [{'name': 'horse', 'bndbox': {'xmax': '222', 'xmin': '129', 'ymax': '152', 'ymin': '66'}, 'difficult': '0', 'occluded': '0', 'pose': 'Right', 'truncated': '0'}, {'name': 'horse', 'bndbox': {'xmax': '500', 'xmin': '261', 'ymax': '333', 'ymin': '1'}, 'difficult': '0', 'occluded': '0', 'pose': 'Frontal', 'truncated': '1'}], 'segmented': '0', 'size': {'depth': '3', 'height': '333', 'width': '500'}, 'source': {'annotation': 'PASCAL VOC2010', 'database

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:865: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Type of targets: <class 'list'>
Content of targets: [{'annotation': {'folder': 'VOC2012', 'filename': '2008_001986.jpg', 'source': {'database': 'The VOC2008 Database', 'annotation': 'PASCAL VOC2008', 'image': 'flickr'}, 'size': {'width': '300', 'height': '500', 'depth': '3'}, 'segmented': '0', 'object': [{'name': 'bicycle', 'pose': 'Left', 'truncated': '0', 'occluded': '1', 'bndbox': {'xmin': '49', 'ymin': '241', 'xmax': '286', 'ymax': '379'}, 'difficult': '0'}, {'name': 'person', 'pose': 'Left', 'truncated': '0', 'occluded': '1', 'bndbox': {'xmin': '69', 'ymin': '190', 'xmax': '236', 'ymax': '354'}, 'difficult': '0'}, {'name': 'person', 'pose': 'Frontal', 'truncated': '1', 'occluded': '1', 'bndbox': {'xmin': '192', 'ymin': '1', 'xmax': '276', 'ymax': '151'}, 'difficult': '0'}, {'name': 'person', 'pose': 'Frontal', 'truncated': '1', 'occluded': '1', 'bndbox': {'xmin': '91', 'ymin': '1', 'xmax': '183', 'ymax': '151'}, 'difficult': '0'}, {'name': 'person', 'pose': 'Frontal', 'truncated':

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:865: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


...

Type of targets: <class 'list'>
Content of targets: [{'annotation': {'filename': '2009_000777.jpg', 'folder': 'VOC2012', 'object': [{'name': 'car', 'bndbox': {'xmax': '478', 'xmin': '5', 'ymax': '369', 'ymin': '32'}, 'difficult': '0', 'occluded': '0', 'pose': 'Left', 'truncated': '0'}, {'name': 'car', 'bndbox': {'xmax': '83', 'xmin': '1', 'ymax': '165', 'ymin': '101'}, 'difficult': '0', 'occluded': '1', 'pose': 'Left', 'truncated': '1'}, {'name': 'car', 'bndbox': {'xmax': '176', 'xmin': '66', 'ymax': '172', 'ymin': '98'}, 'difficult': '0', 'occluded': '1', 'pose': 'Rear', 'truncated': '1'}, {'name': 'person', 'bndbox': {'xmax': '481', 'xmin': '473', 'ymax': '153', 'ymin': '119'}, 'difficult': '0', 'occluded': '0', 'pose': 'Rear', 'truncated': '1'}, {'name': 'car', 'bndbox': {'xmax': '500', 'xmin': '486', 'ymax': '141', 'ymin': '124'}, 'difficult': '0', 'occluded': '0', 'pose': 'Right', 'truncated': '1'}], 'segmented': '0', 'size': {'depth': '3', 'height': '375', 'width': '500'}, 'sourc

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:865: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Type of targets: <class 'list'>
Content of targets: [{'annotation': {'filename': '2009_004984.jpg', 'folder': 'VOC2012', 'object': [{'name': 'bicycle', 'bndbox': {'xmax': '462', 'xmin': '46', 'ymax': '329', 'ymin': '52'}, 'difficult': '0', 'occluded': '0', 'pose': 'Left', 'truncated': '0'}, {'name': 'bottle', 'bndbox': {'xmax': '250', 'xmin': '197', 'ymax': '220', 'ymin': '160'}, 'difficult': '0', 'occluded': '0', 'pose': 'Unspecified', 'truncated': '0'}], 'segmented': '0', 'size': {'depth': '3', 'height': '375', 'width': '500'}, 'source': {'annotation': 'PASCAL VOC2009', 'database': 'The VOC2009 Database', 'image': 'flickr'}}}, {'annotation': {'filename': '2010_002143.jpg', 'folder': 'VOC2012', 'object': [{'name': 'cat', 'bndbox': {'xmax': '284', 'xmin': '136', 'ymax': '316', 'ymin': '23'}, 'difficult': '0', 'occluded': '0', 'pose': 'Unspecified', 'truncated': '0'}], 'segmented': '0', 'size': {'depth': '3', 'height': '375', 'width': '500'}, 'source': {'annotation': 'PASCAL VOC2010', '

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7885b7d65fc0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1477, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1441, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/usr/lib/python3.10/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/usr/lib/python3.10/selectors.py", line 416, in select
    fd_event_list = self._selector.poll(timeout)
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
def val(model, val_loader, device):
    model.to(device)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy